In [1]:
import io
from datetime import date, timedelta, datetime
import numpy as np
import xarray as xr
import requests
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cmocean
import pandas as pd
# Not used directly, but used via xarray
import cfgrib
import tempfile
import tqdm
import wget
import os

from multiprocessing.pool import Pool

In [2]:
!mkdir nrrr file

mkdir: cannot create directory ‘nrrr’: File exists
mkdir: cannot create directory ‘file’: File exists


In [3]:
meta = pd.read_csv('./metadata.csv')

In [4]:
def download_info(x):
    unique_date, sub, cycle = x
    sub = pd.read_json(sub)
    
    blob_container = 'https://noaa-hrrr-bdp-pds.s3.amazonaws.com'
    sector = "conus"
    yesterday = datetime.strptime(unique_date, "%Y-%m-%d")
    # cycle = 0          # noon
    forecast_hour = 1   # offset from cycle time
    product = "wrfsfcf" # 2D surface levels

    # Put it all together
    file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
    url = f"{blob_container}/hrrr.{yesterday:%Y%m%d}/{sector}/{file_path}"

    print(url)
    filename = 'file/' + unique_date + url.split('/')[-1]
    
    wget.download(url, out=filename)
        
    ds = xr.open_dataset(filename, engine='cfgrib', 
                     backend_kwargs={'indexpath':''},
                     filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
    
    sub_infos = []
    for i, row in sub.iterrows():
        lat = row.latitude
        lon = row.longitude + 360

        abslat = np.abs(ds.latitude-lat)
        abslon = np.abs(ds.longitude-lon)
        c = np.maximum(abslon, abslat)

        ([xloc], [yloc]) = np.where(c == np.min(c))

        meta_info = {'id': row['uid']}
        for varname, da in ds.sel(y=xloc, x=yloc).data_vars.items():
            meta_info[da.attrs['long_name']] = da.values.item()
        sub_infos.append(meta_info)

    sub_info = pd.DataFrame(sub_infos)
    sub_info.to_csv('./nrrr/' + unique_date + '.csv')
#     return all_info
#     os.remove(filename)

In [5]:
exist = [x.split('.')[0] for x in os.listdir('./nrrr/')]
len(exist)

6

In [6]:
print(meta[~meta.date.isin(exist)].date.unique().shape)
for unique_date in tqdm.tqdm(meta[~meta.date.isin(exist)].date.unique()):
    fmt = unique_date.replace('-', '')
    sub = meta[meta.date == unique_date].copy()
    try:
        download_info((unique_date, sub.to_json(), 1))
    except:
        pass

(1631,)


  0%|          | 0/1631 [00:00<?, ?it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20171115/conus/hrrr.t01z.wrfsfcf01.grib2


  0%|          | 1/1631 [01:51<50:26:53, 111.42s/it]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20200610/conus/hrrr.t01z.wrfsfcf01.grib2


...

100%|█████████▉| 1629/1631 [17:52:19<00:38, 19.12s/it]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170511/conus/hrrr.t01z.wrfsfcf01.grib2


100%|█████████▉| 1630/1631 [17:52:41<00:20, 20.16s/it]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131208/conus/hrrr.t01z.wrfsfcf01.grib2


100%|██████████| 1631/1631 [17:52:42<00:00, 39.46s/it]


In [7]:
exist = [x.split('.')[0] for x in os.listdir('./nrrr/')]
len(exist)

1290

In [8]:
print(meta[~meta.date.isin(exist)].date.unique().shape)
for unique_date in tqdm.tqdm(meta[~meta.date.isin(exist)].date.unique()):
    fmt = unique_date.replace('-', '')
    sub = meta[meta.date == unique_date].copy()
    try:
        download_info((unique_date, sub.to_json(), 12))
    except:
        pass

(347,)


  0%|          | 0/347 [00:00<?, ?it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140812/conus/hrrr.t12z.wrfsfcf01.grib2


  0%|          | 1/347 [00:00<05:26,  1.06it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131106/conus/hrrr.t12z.wrfsfcf01.grib2


...

 99%|█████████▉| 345/347 [22:55<00:01,  1.06it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20130714/conus/hrrr.t12z.wrfsfcf01.grib2


100%|█████████▉| 346/347 [22:56<00:00,  1.06it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131208/conus/hrrr.t12z.wrfsfcf01.grib2


100%|██████████| 347/347 [22:57<00:00,  3.97s/it]


In [9]:
exist = [x.split('.')[0] for x in os.listdir('./nrrr/')]
len(exist)

1319

In [10]:
print(meta[~meta.date.isin(exist)].date.unique().shape)
for unique_date in tqdm.tqdm(meta[~meta.date.isin(exist)].date.unique()):
    fmt = unique_date.replace('-', '')
    sub = meta[meta.date == unique_date].copy()
    try:
        download_info((unique_date, sub.to_json(), 18))
    except:
        pass

(318,)


  0%|          | 0/318 [00:00<?, ?it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140812/conus/hrrr.t18z.wrfsfcf01.grib2


  0%|          | 1/318 [00:00<05:04,  1.04it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131106/conus/hrrr.t18z.wrfsfcf01.grib2


...

 99%|█████████▉| 316/318 [10:47<00:01,  1.05it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20130714/conus/hrrr.t18z.wrfsfcf01.grib2


100%|█████████▉| 317/318 [10:48<00:00,  1.05it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131208/conus/hrrr.t18z.wrfsfcf01.grib2


100%|██████████| 318/318 [10:49<00:00,  2.04s/it]


In [11]:
exist = [x.split('.')[0] for x in os.listdir('./nrrr/')]
len(exist)

1326

In [12]:
print(meta[~meta.date.isin(exist)].date.unique().shape)
for unique_date in tqdm.tqdm(meta[~meta.date.isin(exist)].date.unique()):
    fmt = unique_date.replace('-', '')
    sub = meta[meta.date == unique_date].copy()
    try:
        download_info((unique_date, sub.to_json(), 22))
    except:
        pass

(311,)


  0%|          | 0/311 [00:00<?, ?it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140812/conus/hrrr.t22z.wrfsfcf01.grib2


  0%|          | 1/311 [00:00<04:47,  1.08it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131106/conus/hrrr.t22z.wrfsfcf01.grib2


...

 99%|█████████▉| 309/311 [07:23<00:01,  1.06it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20130714/conus/hrrr.t22z.wrfsfcf01.grib2


100%|█████████▉| 310/311 [07:24<00:00,  1.06it/s]

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20131208/conus/hrrr.t22z.wrfsfcf01.grib2


100%|██████████| 311/311 [07:25<00:00,  1.43s/it]


## NRRR data start with 2014-08-01 (aws), try search margin case

In [13]:
def download_info_m(x):
    replace_date, unique_date, sub, cycle = x
    sub = pd.read_json(sub)
    
    blob_container = 'https://noaa-hrrr-bdp-pds.s3.amazonaws.com'
    sector = "conus"
    yesterday = datetime.strptime(replace_date, "%Y-%m-%d")
    # cycle = 0          # noon
    forecast_hour = 1   # offset from cycle time
    product = "wrfsfcf" # 2D surface levels

    # Put it all together
    file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
    url = f"{blob_container}/hrrr.{yesterday:%Y%m%d}/{sector}/{file_path}"

    print(url)
    filename = 'file/' + unique_date + url.split('/')[-1]
    
    wget.download(url, out=filename)
        
    ds = xr.open_dataset(filename, engine='cfgrib', 
                     backend_kwargs={'indexpath':''},
                     filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
    
    sub_infos = []
    for i, row in sub.iterrows():
        lat = row.latitude
        lon = row.longitude + 360

        abslat = np.abs(ds.latitude-lat)
        abslon = np.abs(ds.longitude-lon)
        c = np.maximum(abslon, abslat)

        ([xloc], [yloc]) = np.where(c == np.min(c))

        meta_info = {'id': row['uid']}
        for varname, da in ds.sel(y=xloc, x=yloc).data_vars.items():
            meta_info[da.attrs['long_name']] = da.values.item()
        sub_infos.append(meta_info)

    sub_info = pd.DataFrame(sub_infos)
    sub_info.to_csv('./nrrr/' + unique_date + '.csv')


In [14]:
exist = [x.split('.')[0] for x in os.listdir('./nrrr/')]
len(exist)

1328

In [15]:
exist = [x for x in sorted(exist) if x]

In [16]:
rest_todo = meta[~meta.date.isin(exist) & (meta.date > '2014-08-01') & (meta.date < '2014-10-01')].sort_values('date').date.unique()

In [18]:
for r in rest_todo:
    clost = ''
    for est in sorted(exist):
        target = datetime.strptime(r, "%Y-%m-%d")
        candidates = datetime.strptime(est, "%Y-%m-%d")
        if (target - candidates).days <= 0:
            print(f'closest date of {r} is : {clost}')
            # fmt = unique_date.replace('-', '')
            sub = meta[meta.date == r].copy()
            for t in [1, 12, 18, 22]:
                try:
                    download_info_m((clost, r, sub.to_json(), t))
                    break
                except:
                    continue
            break
        else:
            clost = est

closest date of 2014-08-10 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-11 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-12 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-13 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-14 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-15 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-16 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140809/conus/hrrr.t01z.wrfsfcf01.grib2
closest date of 2014-08-17 is : 2014-08-09
https://noaa-hrrr-bdp-pds.s3.amaz